In [41]:
from typing import Any
from enum import Enum

from gekko import GEKKO

BoxType = tuple[float, float, float, float]
InputModule = tuple[BoxType, list[BoxType], list[BoxType], list[BoxType], list[BoxType]]
OptionalMatrix = dict[int, dict[int, float]]

max_ratio = 4

def SMAX(x, y, gekko: GEKKO, tau = 0.01):
    return 0.5 * (x + y + gekko.sqrt( (x - y)**2 + 4 * tau * tau ))
    
def THIN(ratio):
    return ratio + 1/ratio - 1

class Cardinal(Enum):
    NORTH = 0
    SOUTH = 1
    EAST = 2
    WEST = 3

class ModelModule:
    def __init__(self, x: list[Any], y: list[Any], w: list[Any], h: list[Any], gekko: GEKKO, trunk: BoxType):
        self.N: list[int] = []
        self.S: list[int] = []
        self.E: list[int] = []
        self.W: list[int] = []
        self.x: list[Any] = x
        self.y: list[Any] = y
        self.w: list[Any] = w
        self.h: list[Any] = h
        self.c: int = 0
        self.set_trunk(gekko, trunk)
        
    def set_trunk(self, gekko: GEKKO, trunk: BoxType) -> None:
        assert self.c == 0, "!"
        self.c = 1
        self._define_vars(gekko, trunk)
    
    def add_rect_north(self, gekko: GEKKO, rect: BoxType) -> None:
        assert self.c > 0, "!"
        self.N.append(this.c)
        self.c += 1
        xv, yv, wh, hv = this._define_vars(gekko, rect)
        
        # Keep the box attached
        gekko.Equation(yv = self.y[0] + 0.5 * self.h[0] + 0.5 * hv)
        gekko.Equation(xv >= self.x[0] - 0.5 * self.w[0] + 0.5 * wv)
        gekko.Equation(xv <= self.x[0] + 0.5 * self.w[0] - 0.5 * wv)
    
    def add_rect_south(self, gekko: GEKKO, rect: BoxType) -> None:
        assert self.c > 0, "!"
        self.S.append(this.c)
        self.c += 1
        xv, yv, wh, hv = this._define_vars(gekko, rect)
        
        # Keep the box attached
        gekko.Equation(yv = self.y[0] - 0.5 * self.h[0] - 0.5 * hv)
        gekko.Equation(xv >= self.x[0] - 0.5 * self.w[0] + 0.5 * wv)
        gekko.Equation(xv <= self.x[0] + 0.5 * self.w[0] - 0.5 * wv)
        
    def add_rect_east(self, gekko: GEKKO, rect: BoxType) -> None:
        assert self.c > 0, "!"
        self.E.append(this.c)
        self.c += 1
        xv, yv, wh, hv = this._define_vars(gekko, rect)
        
        # Keep the box attached
        gekko.Equation(xv = self.x[0] + 0.5 * self.w[0] + 0.5 * wv)
        gekko.Equation(yv >= self.y[0] - 0.5 * self.h[0] + 0.5 * hv)
        gekko.Equation(yv <= self.y[0] + 0.5 * self.h[0] - 0.5 * hv)
        
    def add_rect_west(self, gekko: GEKKO, rect: BoxType) -> None:
        assert self.c > 0, "!"
        self.W.append(this.c)
        self.c += 1
        xv, yv, wh, hv = this._define_vars(gekko, rect)
        
        # Keep the box attached
        gekko.Equation(xv = self.x[0] - 0.5 * self.w[0] - 0.5 * wv)
        gekko.Equation(yv >= self.y[0] - 0.5 * self.h[0] + 0.5 * hv)
        gekko.Equation(yv <= self.y[0] + 0.5 * self.h[0] - 0.5 * hv)
        
    def _define_vars(self, gekko: GEKKO, rect: BoxType) -> tuple[Any, Any, Any, Any]:
        var_x = gekko.Var()
        var_y = gekko.Var()
        var_w = gekko.Var()
        var_h = gekko.Var()
        var_x.value = rect[0]
        var_y.value = rect[1]
        var_w.value = rect[2]
        var_h.value = rect[3]
        self.x.append(var_x)
        self.y.append(var_y)
        self.w.append(var_w)
        self.h.append(var_h)
        
        # The dimensions must be positive
        gekko.Equation(var_w > 0)
        gekko.Equation(var_h > 0)
        
        # The ratio cannot exceed a maximum value
        gekko.Equation(THIN(var_w / var_h) <= THIN(max_ratio))
        
        return var_x, var_y, var_w, var_h
    
class Model:
    """GEKKO model with variables"""
    gekko: GEKKO
    
    M: list[ModelModule]

    # Model variables
    # (without accurate type hints because GEKKO does not have type hints yet)
    x: list[list[Any]]
    y: list[list[Any]]
    w: list[list[Any]]
    h: list[list[Any]]
        
    def define_module(self, trunk_box: BoxType) -> int:
        x = []
        y = []
        w = []
        h = []
        m = ModelModule(x, y, w, h, self.gekko, trunk_box)
        self.M.append(m)
        self.x.append(x)
        self.y.append(y)
        self.w.append(w)
        self.h.append(h)
        return len(self.M) - 1
    
    def add_rect(self, m: int, box: BoxType, direction: Cardinal) -> None:
        call = self.M[m].add_rect_west
        if direction is Cardinal.NORTH:
            call = self.M[m].add_rect_north
        elif direction is Cardinal.SOUTH:
            call = self.M[m].add_rect_south
        elif direction is Cardinal.EAST:
            call = self.M[m].add_rect_east
        xv, yv, wv, hv = call(trunk_box)
        return len(self.M) - 1
    
    def __init__(self, 
                 M: list[InputModule], 
                 A: list[float], 
                 X: OptionalMatrix, 
                 Y: OptionalMatrix, 
                 W: OptionalMatrix, 
                 H: OptionalMatrix, 
                 Omega):
        assert len(M) == len(A), "M and A need to have the same length!"
        
        self.M = []
        self.x = []
        self.y = []
        self.w = []
        self.h = []
        
        """Constructs the GEKKO object and initializes the model"""
        self.gekko = GEKKO(remote=False)
        
        # Variable definition
        for (trunk, N, S, E, W) in M:
            m = self.define_module(trunk)
            for box in N:
                self.add_rect(m, box, Cardinality.North)
            for box in S:
                self.add_rect(m, box, Cardinality.South)
            for box in E:
                self.add_rect(m, box, Cardinality.East)
            for box in W:
                self.add_rect(m, box, Cardinality.West)
        
        # Minimal area requirements
        for m in range(0, len(A)):
            a = self.w[m][0] * self.h[m][0]
            for j in range(1, len(self.w[m])):
                a += self.w[m][j] * self.h[m][j]
            self.gekko.Equation(a >= A[m])
        
        # No Inter-Module Intersection
        for m in range(0, len(A)):
            for n in range(m+1, len(A)):
                for i in range(0, self.M[m].c):
                    for j in range(0, self.M[n].c):
                        t1 = (self.x[m][i] - self.x[n][j])**2 - 0.25 * (self.w[m][i] + self.w[n][j])
                        t2 = (self.y[m][i] - self.y[n][j])**2 - 0.25 * (self.h[m][i] + self.h[n][j])
                        self.gekko.Equation(SMAX(t1, t2, self.gekko) >= 0)
                        
    def solve(self):
        self.gekko.solve(disp=False)

In [42]:
b1: BoxType = ((2,3,1,5), [], [], [], [])
b2: BoxType = ((2,2,4,4), [], [], [], [])
m = Model([b1, b2], [5, 16], None, None, None, None, None)
m.solve()

In [45]:
(m.x, m.y, m.w, m.h)

([[[2.0]], [[2.0]]],
 [[[3.4907241378]], [[1.7819003832]]],
 [[[1.7517730878]], [[4.178282526]]],
 [[[3.3122574037]], [[4.0359296172]]])